# Speed Skydiving scoring bulk files processor
Uses the `./data` directory as a data lake.  It finds all valid FlySight files in that directory, from 0 to as many as it finds and fit in memory, and process them all.  Summary of "all files" or "all jumps" at the bottom of the document.

In [13]:
from collections import namedtuple
from copy import deepcopy

from ssscoring.constants import BREAKOFF_ALTITUDE
from ssscoring.constants import FT_IN_M
from ssscoring.constants import PERFORMANCE_WINDOW_LENGTH
from ssscoring.fs1 import aggregateResults
from ssscoring.fs1 import convertFlySight2SSScoring
from ssscoring.fs1 import dropNonSkydiveDataFrom
from ssscoring.fs1 import getAllSpeedJumpFilesFrom
from ssscoring.fs1 import getSpeedSkydiveFrom
from ssscoring.fs1 import isValidJump
from ssscoring.fs1 import isValidMinimumAltitude
from ssscoring.fs1 import jumpAnalysisTable
from ssscoring.fs1 import processAllJumpFiles
from ssscoring.fs1 import roundedAggregateResults
from ssscoring.fs1 import totalResultsFrom
from ssscoring.fs1 import validFlySightHeaderIn
from ssscoring.notebook import SPEED_COLORS
from ssscoring.notebook import graphAltitude
from ssscoring.notebook import graphAngle
from ssscoring.notebook import graphJumpResult
from ssscoring.notebook import initializeExtraYRanges
from ssscoring.notebook import initializePlot

import csv
import os
import os.path as path

import bokeh.plotting as bp
import bokeh.models as bm
import ipywidgets as widgets
import pandas as pd

In [14]:
DATA_LAKE_ROOT = './data'

---
## <span style="color: yellow">IMPORTANT</span> - Set the drop zone altitude MSL

Set the value in ft (even though SSScoring uses meters) out of convenience because all DZs in the US, many in Asia and Mexico, and many in Europe express the altitude in feet.

### Some DZs

| Drop zone | Alt (ft) |
|-----------|----------|
| Bay Area Skydiving | 23 |
| Skydive Arizona | 1509 |
| Skydive Chicago | 616 |
| Thai Sky Adventures | 21 |

In [15]:
dropZoneAltMSL = 23

In [16]:
dropZoneAltMSLMeters = dropZoneAltMSL/FT_IN_M
display(widgets.HTML('<h2>DZ Altitude = <span style = "color: green">%7.2f ft</span> (%7.2f m)<h1>' % (dropZoneAltMSL, dropZoneAltMSLMeters)))

HTML(value='<h2>DZ Altitude = <span style = "color: green">  23.00 ft</span> (   7.01 m)<h1>')

In [17]:
jumpFiles = getAllSpeedJumpFilesFrom(DATA_LAKE_ROOT)

In [18]:
jumpResults = processAllJumpFiles(jumpFiles, altitudeDZMeters = dropZoneAltMSLMeters)

### Air density

**<a href='https://aviex.goflexair.com/blog/density-of-air-air-density' target='_blank'>Pilot's Guide to Air Density</a>** - A useful guide with calculations, lots of information.

![](https://ciurana.eu/personal/air-density-partial-table.png)

---
## Results

In [19]:
aggregate = aggregateResults(jumpResults)
sumResults = totalResultsFrom(aggregate)
display(aggregate)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
00-00-00-baseline,480.569538,228.921693,333.417517,421.574180,470.258757,482.706197,24.5,481.932
12-06-24,478.734207,214.992940,326.100976,417.410456,466.748015,482.570024,25.0,481.824


### Rounded results for training log

In [20]:
roundedResults = roundedAggregateResults(jumpResults)

### All jumps

In [21]:
def dumpSimple(table: pd.DataFrame):
    fileName = os.path.join(DATA_LAKE_ROOT, resultRef+'.tsv')
    simpleTable = table.copy()
    simpleTable = simpleTable.drop('speedAngle', axis = 1)
    simpleTable.to_csv(fileName, sep = '\t', index = False)

In [22]:
def displayJumpDataIn(resultsTable: pd.DataFrame):
    table = resultsTable.copy()
    # Experimental
    # For more information on the `interpolate` method and its options, see the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html).
    # Additionally, you can also consider using other libraries like `scipy` which provides more advanced interpolation methods, such as `interp1d` or `griddata`. See the [scipy documentation](https://docs.scipy.org/doc/scipy/reference/interpolate.html) for more information.
    table.vKMh = table.vKMh.apply(round)
    table.hKMh = table.hKMh.apply(round)
    table['altitude (ft)'] = table['altitude (ft)'].apply(round)
    table.netVectorKMh = table.netVectorKMh.apply(round)
    table.index = ['']*len(table)
    display(table)
    # dumpSimple(table)

In [23]:
allJumpsPlot = initializePlot('All jumps in set')
jumpNumber = 0
mixColor = 0
for resultRef in sorted(jumpResults.keys()):
    jumpResult = jumpResults[resultRef]
    if jumpResult.score > 0.0:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - score = %.02f km/h</span></h1>' % (jumpResult.color, resultRef, jumpResult.result, jumpResult.score)        
    else:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s</span></h1>' % (jumpResult.color, resultRef, jumpResult.result)
    display(widgets.HTML(validJumpStatus))

    maxSpeed = jumpResult.maxSpeed
    window = jumpResult.window
    mixColor = (mixColor+1)%len(SPEED_COLORS)
    if jumpResult.score > 0.0:
        display(widgets.HTML('<h3>Max speed = {0:,.0f}; '.format(maxSpeed)+('exit at %d m (%d ft), end scoring window at %d m (%d ft)'%(window.start, 3.2808*window.start, window.end, 3.2808*window.end))))
        if not isValidMinimumAltitude(jumpResult.data.altitudeAGL.max()):
            display(widgets.HTML('<span style="color: yellow"><span style="font-weight: bold">Warning:</span> exit altitude ASL was lower than the minimum scoring altitude according to IPC and USPA.'))
        display(widgets.HTML('</h3>'))
        # TODO: Move this to an in-notebook function or module.
        displayJumpDataIn(jumpResult.table)
        individualPlot = initializePlot(resultRef)
        individualPlot = initializeExtraYRanges(individualPlot, startY = min(jumpResult.data.altitudeAGLFt)-500.0, endY = max(jumpResult.data.altitudeAGLFt)+500.0)
        graphAltitude(individualPlot, jumpResult)
        graphAngle(individualPlot, jumpResult)
        hoverValue = bm.HoverTool(tooltips = [ ('Y-val', '@y{0.00}'), ])
        individualPlot.add_tools(hoverValue)
        graphJumpResult(individualPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[0])
        graphJumpResult(allJumpsPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[mixColor],
                        legend = '%s - %.2f' % (resultRef, jumpResult.score),
                        showIt = False)

HTML(value='<hr><h1><span style="color: #0f0">00-00-00-baseline jump - 🟢 valid - score = 480.57 km/h</span></h…

HTML(value='<h3>Max speed = 482; exit at 4113 m (13495 ft), end scoring window at 1857 m (6094 ft)')

HTML(value='</h3>')

,time,vKMh,hKMh,speedAngle,distanceFromExit,altitude (ft),netVectorKMh
,5.0,187,132,54.7,223.900000,13019,229
,10.0,320,94,73.6,377.000000,11861,333
,15.0,419,46,83.7,471.400000,10128,422
,20.0,470,15,88.2,489.200000,8075,470
,25.0,482,27,86.8,455.678652,6100,483


HTML(value='<hr><h1><span style="color: #0f0">12-06-24 jump - 🟢 valid - score = 478.73 km/h</span></h1>')

HTML(value='<h3>Max speed = 482; exit at 4115 m (13503 ft), end scoring window at 1859 m (6101 ft)')

HTML(value='</h3>')

,time,vKMh,hKMh,speedAngle,distanceFromExit,altitude (ft),netVectorKMh
,5.0,168,134,51.6,205.3,13099,215
,10.0,311,97,72.7,365.6,11981,326
,15.0,412,68,80.7,484.7,10273,417
,20.0,466,23,87.2,538.8,8290,467
,25.0,482,27,86.8,541.6,6115,483


---
## Summary of all speed skydives in the data lake

In [24]:
display(roundedResults)
display(sumResults)
bp.show(allJumpsPlot)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
00-00-00-baseline,481,229,333,422,470,483,25.0,482
12-06-24,479,215,326,417,467,483,25.0,482


,totalSpeed,meanSpeed,maxScore
totalSpeed,959.303745,479.651873,480.569538
